In [1]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import urllib
import time
import tweepy
import pandas as pd
import json

In [2]:

with open('secret.txt') as json_file:
    data = json.load(json_file)
auth = tweepy.OAuthHandler(data['info'][0]['api_key'], data['info'][0]['api_secret'])
auth.set_access_token(data['info'][0]['access_token'], data['info'][0]['access_secret'])

api = tweepy.API(auth)


In [39]:
screen_name='Mahdi_Esr'
user = api.get_user(screen_name =screen_name)
followers_ids = []
for page in tweepy.Cursor(api.followers, screen_name=screen_name).pages():
    followers_ids.extend(page)
    time.sleep(0.1)
    
followings_ids = []
for page in tweepy.Cursor(api.friends, screen_name=screen_name).pages():
    followings_ids.extend(page)
    time.sleep(1)

followers_list_screen_name = [f.screen_name for f in followers_ids]
following_list_screen_name = [f.screen_name for f in followings_ids]
mutual =list(set(followers_list_screen_name).intersection(following_list_screen_name))
mutual_profile_pic_url = [t.profile_image_url_https.replace('_normal.','_200x200.') for t in followers_ids if t.screen_name in mutual]
print('number of mutual friends',len(mutual))

number of mutual friends 70


In [40]:
img = []
for url in mutual_profile_pic_url:
    #print(url)
    img.append(plt.imread(urllib.request.urlopen(url),format= 0))

In [41]:
circle_filter = np.full([200,200,3], True)
for i in range(200):
    for j in range(200):
        if (j-100)**2 + (i-100)**2>100**2:
            circle_filter[i,j] = np.array([False,False,False],dtype=np.bool)


In [42]:
row = 0
clmn = 0
res= np.full([200 * (len(img)//5+1), 200*5,3],fill_value=255, dtype=np.uint8)
for pic in img:
#     print(row,clmn)
    if pic.shape==(200,200,4):
        pic = Image.fromarray(pic)
        pic = pic.convert('RGB')
        pic = np.array(pic)
    r = pic.copy()
    r2 = np.full_like(r, 255)
    r2[circle_filter] = r[circle_filter]
    res[row*200:(row+1)*200, clmn*200:(clmn+1)*200, :] =r2.astype(np.uint8)
    clmn += 1
    if clmn%5==0:
        row +=1
        clmn = 0
plt.imsave(screen_name+'.jpg',res) 